# 🌀 Improved Typhoon Prediction Model v4.1

## Fixes for Flat Prediction Problem

This notebook addresses the **flat prediction issue** where models output near-constant values with minimal variance.

### Key Improvements:
1. **Feature Selection** - Reduce from 8 to 3-4 most predictive features
2. **Lower SVR Epsilon** - Force model to capture variance (0.001-0.1 vs 0.01-0.5)
3. **Leave-One-Out CV** - Better validation for small datasets
4. **Bayesian Models** - BayesianRidge for uncertainty quantification
5. **Lagged Features** - Capture longer-term climate patterns
6. **Diagnostic Tools** - Monitor prediction variance

---
⏱️ **Estimated Runtime**: 5-15 minutes

## 1️⃣ Environment Setup

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
import os
from itertools import product

from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, Matern, WhiteKernel
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, PoissonRegressor, ElasticNet, BayesianRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit, LeaveOneOut, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_regression

print("✅ Environment setup complete!")
print("\n📊 Key Changes from Original:")
print("   • Added BayesianRidge, ElasticNet models")
print("   • Added feature selection (SelectKBest)")
print("   • Added Leave-One-Out cross-validation")
print("   • Lower epsilon values for SVR")

## 2️⃣ Upload Data Files

In [ ]:
from google.colab import files
print("Please upload: typhoon_count.csv and MEI/PDO/IOD/QBO NC files")
uploaded = files.upload()
print(f"\n✅ Uploaded {len(uploaded)} files")

## 3️⃣ Check NC Files

In [ ]:
for f in [f for f in os.listdir('.') if f.endswith('.nc')]:
    ds = xr.open_dataset(f)
    print(f"{f}: variables={list(ds.data_vars)}")
    ds.close()

## 4️⃣ Configuration ⚠️ IMPROVED

### Key Changes:
- **SVR epsilon**: Changed from `[0.01, 0.1, 0.5]` to `[0.001, 0.01, 0.05, 0.1]`
- **Feature selection**: Enabled with max 4 features
- **Leave-One-Out CV**: Enabled for small dataset
- **Added**: BayesianRidge, ElasticNet models
- **Added**: Lagged features (1-2 year lags)

In [ ]:
class Config:
    # Data files - Update if needed
    TYPHOON_DATA_PATH = 'typhoon_count.csv'
    MEI_NC_PATH = 'mei_v2.nc'
    PDO_NC_PATH = 'pdo_ersst_v5.nc'
    IOD_NC_PATH = 'iod_ersst_v5.nc'
    QBO_NC_PATH = 'qbo.nc'

    # NC variable names
    MEI_VAR_NAME = 'mei'
    PDO_VAR_NAME = 'pdo'
    IOD_VAR_NAME = 'iod'
    QBO_VAR_NAME = 'value'
    TIME_VAR_NAME = 'time'

    # Time range
    START_YEAR = 1980
    END_YEAR = 2024
    PREDICT_YEAR = 2025
    TEST_SPLIT_YEAR = 2015

    # ========== KEY FIXES ==========
    # Use Leave-One-Out CV for small datasets (37 samples)
    USE_LOO_CV = True
    CV_N_SPLITS = 5  # Fallback if LOO too slow

    # Feature selection - CRITICAL for small sample size
    USE_FEATURE_SELECTION = True
    MAX_FEATURES = 4  # Reduce from 8 to 4 features

    # Add lagged features for better signal
    USE_LAGGED_FEATURES = True
    LAG_YEARS = [1, 2]

    # Models optimized for small samples
    MODELS_TO_USE = ['SVR', 'BayesianRidge', 'KNN', 'Ridge', 'ElasticNet', 'GaussianProcess']

    # ========== IMPROVED HYPERPARAMETER GRIDS ==========
    PARAM_GRIDS = {
        'SVR': {
            # CRITICAL FIX: Much smaller epsilon values!
            'C': [0.1, 1, 10, 50],
            'epsilon': [0.001, 0.01, 0.05, 0.1],  # Was [0.01, 0.1, 0.5]
            'kernel': ['rbf', 'linear'],
            'gamma': ['scale', 0.001, 0.01, 0.1],
        },
        'BayesianRidge': {
            'alpha_1': [1e-6, 1e-5, 1e-4],
            'alpha_2': [1e-6, 1e-5, 1e-4],
            'lambda_1': [1e-6, 1e-5, 1e-4],
            'lambda_2': [1e-6, 1e-5, 1e-4],
        },
        'GaussianProcess': {
            'alpha': [0.1, 0.5, 1.0],  # Higher alpha for noisy small data
            'kernel_type': ['RBF', 'Matern'],
            'length_scale': [1.0, 2.0, 5.0],
        },
        'KNN': {
            'n_neighbors': [3, 5, 7],
            'weights': ['distance'],  # Distance weighting helps
            'metric': ['euclidean', 'manhattan'],
        },
        'Ridge': {
            'alpha': [0.1, 1.0, 10.0, 100.0],
            'solver': ['svd', 'lsqr'],
        },
        'ElasticNet': {
            'alpha': [0.01, 0.1, 1.0],
            'l1_ratio': [0.2, 0.5, 0.8],
        },
    }

    REGIONS = ['South China Sea', 'Eastern China Sea', 'Japan Sea', 'Yellow Sea']
    USE_TREND_FEATURES = True
    OUTPUT_DIR = 'model_outputs_improved'

os.makedirs(Config.OUTPUT_DIR, exist_ok=True)
print("✅ Configuration complete")
print(f"   Models: {Config.MODELS_TO_USE}")
print(f"   Feature Selection: {Config.USE_FEATURE_SELECTION} (max {Config.MAX_FEATURES} features)")
print(f"   Leave-One-Out CV: {Config.USE_LOO_CV}")
print(f"   SVR epsilon range: {Config.PARAM_GRIDS['SVR']['epsilon']}")

## 5️⃣ Data Loader with Enhanced Features

In [ ]:
class DataLoader:
    def __init__(self, config):
        self.config = config
        self.typhoon_data = None
        self.climate_indices = None

    def load_typhoon_data(self):
        print("Loading typhoon data...")
        self.typhoon_data = pd.read_csv(self.config.TYPHOON_DATA_PATH)
        print(f"  ✓ {len(self.typhoon_data)} records")
        return self.typhoon_data

    def load_climate_index(self, path, var_name, name):
        try:
            ds = xr.open_dataset(path)
            if var_name not in ds.data_vars:
                var_name = list(ds.data_vars)[0]
            df = ds[var_name].to_dataframe().reset_index()
            df['year'] = pd.to_datetime(df['time']).dt.year
            df['month'] = pd.to_datetime(df['time']).dt.month
            df['value'] = df[var_name]
            ds.close()
            return df[['year', 'month', 'value']]
        except Exception as e:
            print(f"  ! {name} load failed: {e}")
            return None

    def load_all_climate_indices(self):
        print("Loading climate indices...")
        self.climate_indices = {}
        for name, path, var in [
            ('MEI', self.config.MEI_NC_PATH, self.config.MEI_VAR_NAME),
            ('PDO', self.config.PDO_NC_PATH, self.config.PDO_VAR_NAME),
            ('IOD', self.config.IOD_NC_PATH, self.config.IOD_VAR_NAME),
            ('QBO', self.config.QBO_NC_PATH, self.config.QBO_VAR_NAME)
        ]:
            data = self.load_climate_index(path, var, name)
            if data is not None:
                self.climate_indices[name] = data
                print(f"  ✓ {name}")
        return self.climate_indices

    def calc_avg(self, data, year, months):
        mask = (data['year'] == year) & (data['month'].isin(months))
        vals = data.loc[mask, 'value']
        return vals.mean() if len(vals) >= 2 else np.nan

    def calc_std(self, data, year, months):
        """Calculate standard deviation for variability features"""
        mask = (data['year'] == year) & (data['month'].isin(months))
        vals = data.loc[mask, 'value']
        return vals.std() if len(vals) >= 2 else np.nan

    def build_feature_matrix(self):
        """Build feature matrix with ENHANCED feature engineering"""
        print("Building feature matrix with enhanced features...")
        years = [y for y in self.typhoon_data['Year'].unique()
                 if self.config.START_YEAR <= y <= self.config.END_YEAR]

        records = []
        for year in sorted(years):
            rec = {'Year': year}
            prev = year - 1

            for idx, data in self.climate_indices.items():
                # Oct-Nov-Dec average (original)
                ond = self.calc_avg(data, prev, [10, 11, 12])
                rec[f'{idx}_OND'] = ond

                # Trend feature (original)
                if self.config.USE_TREND_FEATURES:
                    jas = self.calc_avg(data, prev, [7, 8, 9])
                    rec[f'{idx}_TREND'] = ond - jas if not np.isnan(ond) and not np.isnan(jas) else np.nan

                # NEW: Annual mean
                annual = self.calc_avg(data, prev, list(range(1, 13)))
                rec[f'{idx}_ANNUAL'] = annual

            # NEW: Lagged features (1-2 year lags)
            if self.config.USE_LAGGED_FEATURES:
                for lag in self.config.LAG_YEARS:
                    lag_year = year - 1 - lag
                    for idx, data in self.climate_indices.items():
                        lag_ond = self.calc_avg(data, lag_year, [10, 11, 12])
                        rec[f'{idx}_OND_LAG{lag}'] = lag_ond

            # Current year ENSO (for seasonal prediction)
            if 'MEI' in self.climate_indices:
                rec['MEI_Current_JASO'] = self.calc_avg(
                    self.climate_indices['MEI'], year, [7, 8, 9, 10]
                )

            records.append(rec)

        df = pd.DataFrame(records)

        # Pivot typhoon counts
        pivot = self.typhoon_data.pivot_table(
            index='Year', columns='Region',
            values='Typhoon_Count', aggfunc='sum'
        ).reset_index()

        for col in pivot.columns:
            if col != 'Year':
                pivot = pivot.rename(columns={col: f'Target_{col}'})

        result = df.merge(pivot, on='Year').dropna()
        feature_count = len([c for c in result.columns if c not in ['Year'] and not c.startswith('Target')])
        print(f"  ✓ {len(result)} samples with {feature_count} features")

        return result

print("✅ Data loader module defined")

## 6️⃣ Improved Model Factory

In [ ]:
class ModelFactory:
    @staticmethod
    def create(name, params):
        if name == 'SVR':
            return SVR(
                C=params.get('C', 1),
                epsilon=params.get('epsilon', 0.01),  # Lower default!
                kernel=params.get('kernel', 'rbf'),
                gamma=params.get('gamma', 'scale')
            )
        elif name == 'BayesianRidge':
            return BayesianRidge(
                alpha_1=params.get('alpha_1', 1e-6),
                alpha_2=params.get('alpha_2', 1e-6),
                lambda_1=params.get('lambda_1', 1e-6),
                lambda_2=params.get('lambda_2', 1e-6),
            )
        elif name == 'GaussianProcess':
            kt = params.get('kernel_type', 'RBF')
            ls = params.get('length_scale', 1.0)
            if kt == 'RBF':
                kernel = ConstantKernel(1.0) * RBF(ls) + WhiteKernel(noise_level=0.1)
            else:
                kernel = ConstantKernel(1.0) * Matern(ls, nu=1.5) + WhiteKernel(noise_level=0.1)
            return GaussianProcessRegressor(
                kernel=kernel,
                alpha=params.get('alpha', 0.5),
                random_state=42,
                normalize_y=True
            )
        elif name == 'KNN':
            return KNeighborsRegressor(
                n_neighbors=params.get('n_neighbors', 5),
                weights=params.get('weights', 'distance'),
                metric=params.get('metric', 'euclidean')
            )
        elif name == 'Ridge':
            return Ridge(alpha=params.get('alpha', 1.0))
        elif name == 'ElasticNet':
            return ElasticNet(
                alpha=params.get('alpha', 0.1),
                l1_ratio=params.get('l1_ratio', 0.5),
                max_iter=2000
            )
        elif name == 'RandomForest':
            return RandomForestRegressor(
                n_estimators=params.get('n_estimators', 100),
                max_depth=params.get('max_depth', 3),
                min_samples_split=params.get('min_samples_split', 5),
                random_state=42
            )
        elif name == 'Poisson':
            return PoissonRegressor(
                alpha=params.get('alpha', 1.0),
                max_iter=1000
            )

    @staticmethod
    def get_param_combos(grid):
        if not grid:
            return [{}]
        keys, vals = list(grid.keys()), list(grid.values())
        return [dict(zip(keys, v)) for v in product(*vals)]

print("✅ Model factory defined")

## 7️⃣ Improved Multi-Model System

### Key Improvements:
- **Feature Selection**: Uses mutual information to select top features
- **Leave-One-Out CV**: More reliable for small datasets
- **Variance Monitoring**: Tracks if predictions are too flat
- **Diagnostic Output**: Warns about flat predictions

In [ ]:
class MultiModelSystem:
    def __init__(self, config):
        self.config = config
        self.results = {}
        self.best_models = {}
        self.trained = {}
        self.scalers = {}
        self.feature_selectors = {}
        self.selected_features = {}
        self.correction = {}
        self.feature_cols = None

    def select_features(self, X, y, feature_names, max_features=4):
        """Select most predictive features using mutual information"""
        k = min(max_features, X.shape[1])
        selector = SelectKBest(score_func=mutual_info_regression, k=k)
        selector.fit(X, y)
        mask = selector.get_support()
        selected = [f for f, m in zip(feature_names, mask) if m]
        scores = dict(zip(feature_names, selector.scores_))
        return selector, selected, scores

    def cv_with_loo(self, X, y, model_name, params):
        """Leave-One-Out cross-validation for small datasets"""
        loo = LeaveOneOut()
        predictions = []
        actuals = []

        for train_idx, test_idx in loo.split(X):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            scaler = StandardScaler()
            X_train_s = scaler.fit_transform(X_train)
            X_test_s = scaler.transform(X_test)

            y_train_fit = np.maximum(y_train, 0.1) if model_name == 'Poisson' else y_train

            try:
                model = ModelFactory.create(model_name, params)
                model.fit(X_train_s, y_train_fit)
                pred = np.maximum(model.predict(X_test_s), 0)[0]
                predictions.append(pred)
                actuals.append(y_test[0])
            except Exception as e:
                return 999, 999

        predictions = np.array(predictions)
        actuals = np.array(actuals)
        rmse = np.sqrt(mean_squared_error(actuals, predictions))
        mae = mean_absolute_error(actuals, predictions)

        return rmse, mae

    def cv_with_tscv(self, X, y, model_name, params, n_splits=5):
        """Time series cross-validation fallback"""
        tscv = TimeSeriesSplit(n_splits=n_splits)
        rmses = []

        for train_idx, val_idx in tscv.split(X):
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X[train_idx])
            X_val = scaler.transform(X[val_idx])
            y_train = np.maximum(y[train_idx], 0.1) if model_name == 'Poisson' else y[train_idx]

            try:
                model = ModelFactory.create(model_name, params)
                model.fit(X_train, y_train)
                pred = np.maximum(model.predict(X_val), 0)
                rmses.append(np.sqrt(mean_squared_error(y[val_idx], pred)))
            except:
                rmses.append(999)

        return np.mean(rmses), np.std(rmses)

    def search_best_params(self, X, y, model_name, grid):
        """Search for best hyperparameters"""
        combos = ModelFactory.get_param_combos(grid)
        best_rmse = float('inf')
        best_params = {}

        for params in combos:
            if self.config.USE_LOO_CV:
                rmse, _ = self.cv_with_loo(X, y, model_name, params)
            else:
                rmse, _ = self.cv_with_tscv(X, y, model_name, params, self.config.CV_N_SPLITS)

            if rmse < best_rmse:
                best_rmse = rmse
                best_params = params

        return best_params, best_rmse

    def train_region(self, data, region, feat_cols):
        """Train models for a specific region with improvements"""
        target = f'Target_{region}'
        if target not in data.columns:
            print(f"  ! Target column {target} not found")
            return None

        X = data[feat_cols].values
        y = data[target].values
        years = data['Year'].values

        train_mask = years < self.config.TEST_SPLIT_YEAR
        test_mask = years >= self.config.TEST_SPLIT_YEAR

        X_train, X_test = X[train_mask], X[test_mask]
        y_train, y_test = y[train_mask], y[test_mask]

        print(f"\n  === {region} ===")
        print(f"      Training: {len(y_train)} samples, Test: {len(y_test)} samples")
        print(f"      Target mean: {y_train.mean():.1f}, std: {y_train.std():.1f}")

        # Feature selection
        if self.config.USE_FEATURE_SELECTION and len(feat_cols) > self.config.MAX_FEATURES:
            selector, selected, scores = self.select_features(
                X_train, y_train, feat_cols, self.config.MAX_FEATURES
            )
            self.feature_selectors[region] = selector
            self.selected_features[region] = selected
            X_train = selector.transform(X_train)
            X_test = selector.transform(X_test)
            print(f"      Selected features: {selected}")
        else:
            self.selected_features[region] = feat_cols

        # Scale
        scaler = StandardScaler()
        X_train_s = scaler.fit_transform(X_train)
        X_test_s = scaler.transform(X_test)
        self.scalers[region] = scaler

        # Search for best model
        model_results = {}
        for model_name in self.config.MODELS_TO_USE:
            print(f"    {model_name}...", end=" ")
            grid = self.config.PARAM_GRIDS.get(model_name, {})

            best_params, cv_rmse = self.search_best_params(X_train, y_train, model_name, grid)
            print(f"CV RMSE={cv_rmse:.2f}")

            model_results[model_name] = {
                'params': best_params,
                'cv_rmse': cv_rmse
            }

        # Select best model
        best_model_name = min(model_results, key=lambda m: model_results[m]['cv_rmse'])
        best_params = model_results[best_model_name]['params']
        print(f"  🏆 Best model: {best_model_name}")
        if best_model_name == 'SVR':
            print(f"      SVR epsilon: {best_params.get('epsilon', 'default')}")

        # Train final model
        y_train_fit = np.maximum(y_train, 0.1) if best_model_name == 'Poisson' else y_train
        final_model = ModelFactory.create(best_model_name, best_params)
        final_model.fit(X_train_s, y_train_fit)

        # Predictions
        y_train_pred = np.maximum(final_model.predict(X_train_s), 0)
        y_test_pred = np.maximum(final_model.predict(X_test_s), 0)

        # Metrics
        test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
        test_r2 = r2_score(y_test, y_test_pred)
        train_r2 = r2_score(y_train, y_train_pred)

        # Check for flat predictions (DIAGNOSTIC)
        pred_std = np.std(y_test_pred)
        actual_std = np.std(y_test)
        variance_ratio = pred_std / actual_std if actual_std > 0 else 0

        print(f"     Train R²={train_r2:.2f}, Test RMSE={test_rmse:.2f}, Test R²={test_r2:.2f}")
        print(f"     Prediction std={pred_std:.2f}, Actual std={actual_std:.2f}, Variance ratio={variance_ratio:.2f}")

        if variance_ratio < 0.3:
            print(f"     ⚠️ WARNING: Predictions still too flat!")
        elif variance_ratio < 0.6:
            print(f"     ⚠️ CAUTION: Predictions under-capture variance")
        else:
            print(f"     ✓ Variance capture looks reasonable")

        # Store results
        self.best_models[region] = best_model_name
        self.trained[region] = {best_model_name: final_model}

        # Train all models for comparison
        for mn in self.config.MODELS_TO_USE:
            if mn not in self.trained[region]:
                try:
                    yt = np.maximum(y_train, 0.1) if mn == 'Poisson' else y_train
                    m = ModelFactory.create(mn, model_results[mn]['params'])
                    m.fit(X_train_s, yt)
                    self.trained[region][mn] = m
                    pred = np.maximum(m.predict(X_test_s), 0)
                    model_results[mn]['test_rmse'] = np.sqrt(mean_squared_error(y_test, pred))
                    model_results[mn]['test_r2'] = r2_score(y_test, pred)
                except:
                    model_results[mn]['test_rmse'] = 999
                    model_results[mn]['test_r2'] = -999

        model_results[best_model_name]['test_rmse'] = test_rmse
        model_results[best_model_name]['test_r2'] = test_r2

        # ENSO correction
        if 'MEI_Current_JASO' in data.columns:
            mei = data.loc[train_mask, 'MEI_Current_JASO'].values
            res = y_train - y_train_pred
            valid = ~np.isnan(mei)
            if valid.sum() > 5:
                slope, inter = np.polyfit(mei[valid], res[valid], 1)
                self.correction[region] = {'slope': slope, 'intercept': inter}

        self.results[region] = {
            'models': model_results,
            'best': best_model_name,
            'best_params': best_params,
            'test_rmse': test_rmse,
            'test_r2': test_r2,
            'train_r2': train_r2,
            'y_train': y_train,
            'y_train_pred': y_train_pred,
            'y_test': y_test,
            'y_test_pred': y_test_pred,
            'years_train': years[train_mask],
            'years_test': years[test_mask],
            'pred_std': pred_std,
            'variance_ratio': variance_ratio,
        }

    def train_all(self, data):
        """Train models for all regions"""
        self.feature_cols = [c for c in data.columns
                           if c not in ['Year'] and not c.startswith('Target')]

        print(f"\nTotal available features: {len(self.feature_cols)}")
        print(f"Features: {self.feature_cols}")

        for region in self.config.REGIONS:
            self.train_region(data, region, self.feature_cols)

    def predict(self, features, enso=None):
        """Make predictions for new data"""
        preds = {}
        for region, mn in self.best_models.items():
            # Get feature values
            if region in self.feature_selectors:
                X = np.array([[features[c] for c in self.feature_cols]])
                X = self.feature_selectors[region].transform(X)
            else:
                selected = self.selected_features[region]
                X = np.array([[features[c] for c in selected]])

            Xs = self.scalers[region].transform(X)
            base = float(np.maximum(self.trained[region][mn].predict(Xs), 0)[0])

            corr = 0
            if enso and region in self.correction:
                corr = self.correction[region]['slope'] * enso + self.correction[region]['intercept']

            final = max(0, base + corr)
            preds[region] = {'model': mn, 'base': base, 'corr': corr, 'final': round(final)}

        return preds

    def comparison_table(self):
        """Generate comparison table"""
        rows = []
        for region, res in self.results.items():
            for mn, mr in res['models'].items():
                rows.append({
                    'Region': region,
                    'Model': mn,
                    'CV_RMSE': mr['cv_rmse'],
                    'Test_RMSE': mr.get('test_rmse', np.nan),
                    'Test_R2': mr.get('test_r2', np.nan),
                    'Best': '★' if mn == res['best'] else ''
                })
        return pd.DataFrame(rows)

print("✅ Multi-model system defined")

## 8️⃣ Load Data and Build Features

In [ ]:
# Load data
loader = DataLoader(Config)
loader.load_typhoon_data()
loader.load_all_climate_indices()
data = loader.build_feature_matrix()

print("\n" + "="*60)
print("Sample data:")
display(data.head())

print(f"\n📊 Dataset Summary:")
print(f"   Total samples: {len(data)}")
print(f"   Training samples (< {Config.TEST_SPLIT_YEAR}): {len(data[data['Year'] < Config.TEST_SPLIT_YEAR])}")
print(f"   Test samples (>= {Config.TEST_SPLIT_YEAR}): {len(data[data['Year'] >= Config.TEST_SPLIT_YEAR])}")

## 9️⃣ Train Models

In [ ]:
print("⏳ Starting training (this may take several minutes)...")
print("   Using Leave-One-Out CV for reliable estimates on small data\n")

system = MultiModelSystem(Config)
system.train_all(data)

print("\n" + "="*60)
print("✅ Training complete!")

## 🔟 Results Analysis

In [ ]:
# Model comparison table
comparison = system.comparison_table()
print("\n📊 Model Comparison by Region:")
display(comparison)

# Save results
comparison.to_csv(f'{Config.OUTPUT_DIR}/model_comparison_improved.csv', index=False)
print(f"\n✓ Results saved to {Config.OUTPUT_DIR}/")

## 1️⃣1️⃣ Visualization

In [ ]:
# Plot actual vs predicted
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, (region, res) in enumerate(system.results.items()):
    ax = axes[idx]

    # Combine train and test
    all_years = np.concatenate([res['years_train'], res['years_test']])
    all_actual = np.concatenate([res['y_train'], res['y_test']])
    all_pred = np.concatenate([res['y_train_pred'], res['y_test_pred']])

    # Plot actual
    ax.plot(all_years, all_actual, 'b-o', label='Actual', linewidth=2, markersize=4)

    # Plot predictions
    ax.plot(res['years_train'], res['y_train_pred'], 'g--s',
           label='Train Pred', linewidth=1.5, markersize=3, alpha=0.7)
    ax.plot(res['years_test'], res['y_test_pred'], 'r--^',
           label='Test Pred', linewidth=2, markersize=5)

    # Add train/test split line
    ax.axvline(x=Config.TEST_SPLIT_YEAR - 0.5, color='gray',
              linestyle=':', linewidth=2, label='Train/Test Split')

    # Add mean line for reference
    mean_val = np.mean(all_actual)
    ax.axhline(y=mean_val, color='orange', linestyle='--',
              alpha=0.5, label=f'Mean ({mean_val:.1f})')

    ax.set_xlabel('Year')
    ax.set_ylabel('Typhoon Count')
    ax.set_title(f'{region}\nBest: {res["best"]} | R²={res["test_r2"]:.2f} | Var.Ratio={res["variance_ratio"]:.2f}')
    ax.legend(loc='upper right', fontsize=8)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{Config.OUTPUT_DIR}/predictions_improved.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✓ Plot saved to {Config.OUTPUT_DIR}/predictions_improved.png")

## 1️⃣2️⃣ Diagnostic Summary

In [ ]:
print("="*60)
print("📊 DIAGNOSTIC SUMMARY")
print("="*60)

for region, res in system.results.items():
    print(f"\n🌊 {region}:")
    print(f"   Best Model: {res['best']}")
    print(f"   Parameters: {res['best_params']}")
    print(f"   Train R²: {res['train_r2']:.3f}")
    print(f"   Test R²: {res['test_r2']:.3f}")
    print(f"   Test RMSE: {res['test_rmse']:.3f}")
    print(f"   Prediction Variance Ratio: {res['variance_ratio']:.3f}")

    if res['variance_ratio'] < 0.3:
        print(f"   ⚠️ ISSUE: Predictions still too flat!")
        print(f"      Consider: More data, different features, or simpler baseline model")
    elif res['test_r2'] < 0:
        print(f"   ⚠️ ISSUE: Model worse than mean prediction!")
    else:
        print(f"   ✓ Model shows reasonable performance")

    if region in system.selected_features:
        print(f"   Selected Features: {system.selected_features[region]}")

print("\n" + "="*60)
print("COMPARISON: Original vs Improved")
print("="*60)
print("""
Original Issues:
  • SVR epsilon: [0.01, 0.1, 0.5] → too high, ignored variance
  • 8 features for 37 samples → overfitting
  • Negative R² values → worse than mean prediction

Improvements Applied:
  • SVR epsilon: [0.001, 0.01, 0.05, 0.1] → captures variance
  • Feature selection: 4 best features → better ratio
  • Leave-One-Out CV → reliable for small data
  • BayesianRidge → handles uncertainty better
  • Lagged features → more signal
""")

## 1️⃣3️⃣ Make 2025 Prediction (Optional)

In [ ]:
# Get latest feature values for 2025 prediction
# This uses the most recent year's data as proxy

latest_year = data['Year'].max()
latest_row = data[data['Year'] == latest_year].iloc[0]

# Build feature dict
features_2025 = {}
for col in system.feature_cols:
    if col in latest_row:
        features_2025[col] = latest_row[col]

# Get ENSO value if available
enso_val = latest_row.get('MEI_Current_JASO', None)

print(f"\n🌀 Predictions for {Config.PREDICT_YEAR}:")
print(f"   (Using {latest_year} climate features as proxy)\n")

predictions = system.predict(features_2025, enso=enso_val)

for region, pred in predictions.items():
    print(f"   {region}:")
    print(f"      Model: {pred['model']}")
    print(f"      Base prediction: {pred['base']:.1f}")
    print(f"      ENSO correction: {pred['corr']:.1f}")
    print(f"      Final prediction: {pred['final']} typhoons")
    print()

## 1️⃣4️⃣ Download Results

In [ ]:
from google.colab import files

# Download comparison CSV
files.download(f'{Config.OUTPUT_DIR}/model_comparison_improved.csv')

# Download plot
files.download(f'{Config.OUTPUT_DIR}/predictions_improved.png')

print("✅ Files downloaded!")

---

## 📝 Notes on Flat Prediction Fix

### Why predictions were flat:
1. **High SVR epsilon** (0.1-0.5) ignored small variations
2. **Too many features** (8) for sample size (37)
3. **Over-regularization** from small data

### How this notebook fixes it:
1. **Lower epsilon** (0.001-0.1) forces SVR to capture variance
2. **Feature selection** reduces to 4 best features
3. **Leave-One-Out CV** gives reliable estimates
4. **Bayesian models** handle uncertainty better

### If predictions are still flat:
- The signal in climate indices may be weak
- Try different feature combinations
- Consider ensemble averaging
- Accept that some uncertainty is inherent with 37 samples